Instalación de driver `pymongo`:
https://pymongo.readthedocs.io/en/stable/installation.html

In [1]:
import pandas as pd
import numpy as np
import datetime
import pprint
from bson.objectid import ObjectId
import pymongo
from pymongo import GEOSPHERE
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

## Conexión con el servidor de MongoDB

In [2]:
# es importante mapear al puerto usando -p 27017:27017 al construir el contenedor o en el docker-compose.yml
conn_str = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn_str, server_api=ServerApi('1'), serverSelectionTimeoutMS=5000)

#Alternativa:
# client = MongoClient('localhost', 27017)

# Probando conexión
try:
    print(client.server_info())
except Exception:
    print("Unable to connect to the server.")

{'version': '5.0.8', 'gitVersion': 'c87e1c23421bf79614baf500fda6622bd90f674e', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [5, 0, 8, 0], 'openssl': {'running': 'OpenSSL 1.1.1f  31 Mar 2020', 'compiled': 'OpenSSL 1.1.1f  31 Mar 2020'}, 'buildEnvironment': {'distmod': 'ubuntu2004', 'distarch': 'aarch64', 'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.5.0', 'ccflags': '-Werror -include mongo/platform/basic.h -fasynchronous-unwind-tables -ggdb -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -fno-omit-frame-pointer -fno-strict-aliasing -O2 -march=armv8.2-a -mtune=generic -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -Wa,--nocompress-debug-sections -moutline-atomics', 'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 8.5.0', 'cxxflags': '-Woverloaded-virtual -Wno-maybe-uninitialize

## Creación de la base

In [3]:
# Borrar base
client.drop_database('BDBicis')

db = client['BDBicis']

### Funciones para añadir registros

In [4]:
# Colección: Estaciones
# Estamos usando id propios
def carga_estacion( id:int, nombre:str, longitud, latitud):
    exists = db.Estaciones.find_one({'_id': id})
    if exists:
        return 0

    estacion = {'_id': id,
                'nombre_estacion': nombre,
                'ubicacion': {'type': "point", 'coordinates' : [longitud, latitud]}}
    
    estacion_id = db.Estaciones.insert_one(estacion).inserted_id

    return estacion_id

In [5]:
# Colección: Rutas
def carga_ruta( origen_id:int, destino_id:int, tiempo_promedio:float):
    
    exists = db.Rutas.find_one({'id_origen': origen_id, 'id_destino': destino_id})
    if exists:
        return 0
    
    ruta = {'id_origen': origen_id,
            'id_destino': destino_id,
            'tiempo_promedio': tiempo_promedio}

    ruta_id = db.Rutas.insert_one(ruta).inserted_id

    return ruta_id

In [6]:
# Colección: Viajes
def carga_viaje( origen_id:int, destino_id:int, salida:datetime, llegada:datetime, usuario):
    duracion = (llegada - salida).seconds
    viaje = {'id_origen': origen_id,
             'id_destino': destino_id,
             'hora_salida': salida,
             'hora_llegada': llegada,
             'duracion': duracion,
             'usuario': usuario}
    
    viaje_id = db.Viajes.insert_one(viaje).inserted_id
    
    return viaje_id

In [7]:
# Colección: Usuarios
def crear_usuario( nombre:str, nombre_lugar:str, longitud_lugar, latitud_lugar):
    existe = db.Usuarios.find_one({'nombre_usuario': nombre})
    if existe:
        return existe['_id']

    estaciones = db.Estaciones.aggregate([
    { 
        "$geoNear": {
            "near": [ longitud_lugar , latitud_lugar],
            "distanceField": "distancia", 
            "maxDistance": 50000,
            "spherical": True
        }
    },
    {
        "$limit": 3 
    }])
    estaciones_mas_cercanas = []
    for estacion in estaciones:
        estaciones_mas_cercanas.append(estacion)

    usuario = {'nombre_usuario' : nombre,
               'lugares': [{'nombre_lugar': nombre_lugar,
                           'ubicacion': {'type': "point", 
                                        'coordinates' : [longitud_lugar, latitud_lugar]},
                           'estaciones': estaciones_mas_cercanas}]
                }
    
    usuario_id = db.Usuarios.insert_one(usuario).inserted_id
    return usuario_id


### Carga de los datos

In [8]:
# Borrar base y volver a crear
client.drop_database('BDBicis')

db = client['BDBicis']

In [9]:
# Estaciones
estaciones = pd.read_csv('data/estaciones.csv')
estaciones = estaciones[['start station id', 'start station name', 'start station longitude', 'start station latitude']]

for estacion in estaciones.values:
    carga_estacion(estacion[0], estacion[1], estacion[2], estacion[3]) 

In [10]:
db.Estaciones.create_index( [("ubicacion.coordinates", pymongo.GEOSPHERE)] )   

'ubicacion.coordinates_2dsphere'

In [11]:
#### OJO: esta celda tarda en mi compu como 18 minutos en cargar todos los registros en rutas.values

# Rutas
rutas = pd.read_csv('data/viajes.csv')

for ruta in rutas.values[:10000]: #Cargar 10,000 valores toma menos de 1 minuto
    carga_ruta(ruta[0], ruta[1], ruta[2])

# Queries

**Punto 1**
- Verificar si usuario existe
- Dar de alta usuario
- Añadir/actualizar lugar favorito
- Consultar lista de lugares favoritos

**Punto 2**
- Consultar estaciones más cercanas a cada lugar favorito

**Punto 3**
- Dado un tiempo y una estación, sugerir destino.

**Punto 4**
- Dado un tiempo y una estación, sugerir viaje redondo.

**Extras**
- Consultar lista de viajes realizados.

### Punto 1

In [12]:
# Dar de alta usuario: crear_usuario()

# Verificar si usuario existe
def existe_usuario(usuario:str):
    existe = db.Usuarios.find_one({'nombre_usuario': usuario})
    if existe:
        return existe['_id']
    else:
        return False

# Regresa lista de lugares guardados del usuario
def lugares_guardados(usuario:str):
    respuesta = db.Usuarios.find_one({'nombre_usuario': usuario}, {'lugares':1})
    lugares_guardados = []
    for lugar in respuesta['lugares']:
        nombre = lugar['nombre_lugar']
        coordenadas = lugar['ubicacion']['coordinates']
        lugares_guardados.append((nombre, coordenadas))

    return lugares_guardados

# Determina si un nombre de lugar ya ha sido ocupado por ese usuario
def existe_lugar(usuario:str, nombre_lugar:str):
    existe = db.Usuarios.find_one({'nombre_usuario': usuario, 'lugares.nombre_lugar':nombre_lugar})
    if existe:
        return existe['lugares'][0]['ubicacion']['coordinates']
    return False


# Añade un nuevo lugar, si es que este no existe
def nuevo_lugar(usuario:str, nombre_lugar:str, longitud_lugar, latitud_lugar):
    existe = db.Usuarios.find_one({'nombre_usuario': usuario, 'lugares.nombre_lugar':nombre_lugar})
    if existe:
        return 0

    estaciones = db.Estaciones.aggregate([
    { 
        "$geoNear": {
            "near": [ longitud_lugar , latitud_lugar],
            "distanceField": "distancia", 
            "maxDistance": 50000,
            "spherical": True
        }
    },
    {
        "$limit": 3
    }])
    estaciones_mas_cercanas = []
    for estacion in estaciones:
        estaciones_mas_cercanas.append(estacion)
    
    lugares = db.Usuarios.find_one({'nombre_usuario':usuario})['lugares']

    nuevo_lugar = {'nombre_lugar': nombre_lugar,
                    'ubicacion': {'type': "point", 
                                        'coordinates' : [longitud_lugar, latitud_lugar]},
                    'estaciones': estaciones_mas_cercanas}
    
    lugares.append(nuevo_lugar)

    db.Usuarios.update_one({'nombre_usuario':usuario}, {'$set':{'lugares': lugares}})
    return True


### Punto 2

In [13]:
# Estaciones más cercanas a un lugar guardado
def estaciones_mas_cercanas(usuario:str, nombre_lugar:str):
    existe = db.Usuarios.find_one({'nombre_usuario': usuario, 'lugares.nombre_lugar':nombre_lugar})
    if existe:
        return existe['lugares'][0]['estaciones']
    return None

# Estaciones más cercanas a una ubicación cualquiera
def estaciones_mas_cercanas_loc(longitud:float, latitud:float, limit:int=3):
    estaciones = db.Estaciones.aggregate([
    { 
        "$geoNear": {
            "near": [ longitud , latitud],
            "distanceField": "distancia", 
            "maxDistance": 50000,
            "spherical": True
        }
    },
    {
        "$limit": limit
    }])
    estaciones_mas_cercanas = []
    for estacion in estaciones:
        estaciones_mas_cercanas.append(estacion)
    return estaciones_mas_cercanas

### Punto 3

In [14]:
# Sugerir ruta dada estación y tiempo de viaje (en segundos)
def ruta_desde_estacion( id_origen:int, tiempo_viaje:int):
    respuesta = db.Rutas.find({'id_origen':id_origen,
                               'id_destino': {'$ne': id_origen},
                               'tiempo_promedio':{ '$gt': tiempo_viaje-3600, '$lt': tiempo_viaje+3600}})
    
    respuesta = pd.DataFrame(respuesta)

    if len(respuesta) == 0:
        return None
        
    respuesta['dif_tiempo'] = abs(respuesta['tiempo_promedio'] - tiempo_viaje )
    respuesta =  respuesta.sort_values(by='dif_tiempo')

    return respuesta.head(10)
        

### Punto 4

In [15]:
# Viaje redondo
def viaje_redondo(id_origen:int, tiempo_viaje:int):
    
    viaje_ida = db.Rutas.find({'id_origen':id_origen,
                               'id_destino': {'$ne': id_origen},
                               'tiempo_promedio':{ '$gt': tiempo_viaje/2-3600, '$lt': tiempo_viaje/2+3600}})
    viaje_vuelta = db.Rutas.find({'id_origen':{'$ne': id_origen},
                               'id_destino': id_origen,
                               'tiempo_promedio':{ '$gt': tiempo_viaje/2-3600, '$lt': tiempo_viaje/2+3600}})
    
    viaje_ida = pd.DataFrame(viaje_ida)[['id_origen', 'id_destino', 'tiempo_promedio']]
    viaje_ida.columns = ['origen', 'punto_medio', 'tiempo_ida']

    viaje_vuelta = pd.DataFrame(viaje_vuelta)[['id_origen','id_destino','tiempo_promedio']]
    viaje_vuelta.columns = ['punto_medio', 'destino', 'tiempo_vuelta']
    
    viaje_redondo = viaje_ida.merge(right=viaje_vuelta, how='inner', on='punto_medio')
    viaje_redondo['tiempo_promedio'] = viaje_redondo['tiempo_ida'] + viaje_redondo['tiempo_vuelta']

    viaje_redondo['dif_tiempo'] = abs(viaje_redondo['tiempo_promedio'] - tiempo_viaje )
    viaje_redondo =  viaje_redondo.sort_values(by='dif_tiempo')

    return viaje_redondo.head(10)

# Testing

### Creación de usuarios y actualización de lugares guardados

In [16]:
# Lista de usuarios
lista_usuarios = db.Usuarios.find()
for usuario in lista_usuarios:
    print(usuario['nombre_usuario'])

In [17]:
crear_usuario('Marcela', 'Casa',-73.97032527, 40.75323099 )

ObjectId('626f6e102a7e424e77da2bba')

In [18]:
lugares_guardados('Marcela')

[('Casa', [-73.97032527, 40.75323099])]

In [19]:
existe_lugar('Marcela', 'Trabajo')

False

In [20]:
nuevo_lugar('Marcela', 'Trabajo', -73.97032227, 40.75323489)

True

In [21]:
lugares_guardados('Marcela')

[('Casa', [-73.97032527, 40.75323099]),
 ('Trabajo', [-73.97032227, 40.75323489])]

In [22]:
existe_usuario('Marcela')

ObjectId('626f6e102a7e424e77da2bba')

In [23]:
existe_usuario('Diego')

False

### Estaciones más cercanas

In [24]:
print(estaciones_mas_cercanas('Marcela', 'Escuela'))

None


In [25]:
estaciones_mas_cercanas('Marcela', 'Casa')

[{'_id': 164,
  'nombre_estacion': 'E 47 St & 2 Ave',
  'ubicacion': {'type': 'point', 'coordinates': [-73.97032517, 40.75323098]},
  'distancia': 1.3336063451835308e-09},
 {'_id': 228,
  'nombre_estacion': 'E 48 St & 3 Ave',
  'ubicacion': {'type': 'point',
   'coordinates': [-73.971878855, 40.7546011026]},
  'distancia': 3.152354973152445e-05},
 {'_id': 440,
  'nombre_estacion': 'E 45 St & 3 Ave',
  'ubicacion': {'type': 'point', 'coordinates': [-73.97282625, 40.75255434]},
  'distancia': 3.511219014303481e-05}]

### Sugerir viajes

In [26]:
# Aquí puede devolver None si no se han cargado todas las rutas
ruta_desde_estacion(164, 600)

,_id,id_origen,id_destino,tiempo_promedio,dif_tiempo
15,626f6e022a7e424e77da1ca3,164.0,153.0,600.500000,0.500000
241,626f6e022a7e424e77da1d88,164.0,507.0,602.394030,2.394030
21,626f6e022a7e424e77da1caa,164.0,173.0,609.108911,9.108911
268,626f6e022a7e424e77da1da3,164.0,536.0,609.827957,9.827957
286,626f6e032a7e424e77da1db5,164.0,2022.0,587.852174,12.147826
214,626f6e022a7e424e77da1d6c,164.0,476.0,614.360000,14.360000
65,626f6e022a7e424e77da1cd6,164.0,290.0,619.937729,19.937729
8,626f6e022a7e424e77da1c9c,164.0,137.0,575.967742,24.032258
210,626f6e022a7e424e77da1d68,164.0,472.0,625.703390,25.703390
272,626f6e022a7e424e77da1da7,164.0,540.0,625.752577,25.752577


In [27]:
ruta_desde_estacion(72, 600)

,_id,id_origen,id_destino,tiempo_promedio,dif_tiempo
173,626f6df22a7e424e77da055a,72.0,423.0,611.975000,11.975000
248,626f6df22a7e424e77da05a7,72.0,508.0,583.680000,16.320000
236,626f6df22a7e424e77da059b,72.0,495.0,625.405797,25.405797
230,626f6df22a7e424e77da0595,72.0,488.0,567.171548,32.828452
20,626f6df22a7e424e77da04c0,72.0,173.0,639.500000,39.500000
260,626f6df22a7e424e77da05b3,72.0,520.0,641.927536,41.927536
235,626f6df22a7e424e77da059a,72.0,494.0,663.933333,63.933333
194,626f6df22a7e424e77da0571,72.0,450.0,534.301282,65.698718
221,626f6df22a7e424e77da058c,72.0,479.0,528.299492,71.700508
211,626f6df22a7e424e77da0582,72.0,469.0,522.524096,77.475904


In [28]:
viaje_redondo(72, 900)

,origen,punto_medio,tiempo_ida,destino,tiempo_vuelta,tiempo_promedio,dif_tiempo
19,72.0,173.0,639.500000,72.0,564.548276,1204.048276,304.048276
14,72.0,160.0,717.785714,72.0,723.848485,1441.634199,541.634199
6,72.0,137.0,749.234043,72.0,790.696970,1539.931012,639.931012
28,72.0,228.0,824.190476,72.0,774.014493,1598.204969,698.204969
12,72.0,153.0,877.152941,72.0,744.254545,1621.407487,721.407487
16,72.0,164.0,855.727273,72.0,857.781250,1713.508523,813.508523
25,72.0,223.0,770.923077,72.0,1022.133333,1793.056410,893.056410
2,72.0,83.0,923.157895,72.0,981.687500,1904.845395,1004.845395
9,72.0,150.0,1288.636364,72.0,646.250000,1934.886364,1034.886364
22,72.0,212.0,974.658228,72.0,1026.066327,2000.724554,1100.724554
